In [1]:
from typing import List
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text
import os
import json
from tqdm import tqdm

Онтология из репозитория

In [2]:
ont = {
	"title": "Movie Ontology",
	"id": "ont_1_movie",
	"concepts": [
		{"qid": "Q5", "label": "human"},
		{"qid": "Q515", "label": "city"},
		{"qid": "Q6256", "label": "country"},
		{"qid": "Q11424", "label": "film"},
		{"qid": "Q201658", "label": "film genre"},
		{"qid": "Q483394", "label": "genre"},
		{"qid": "Q1762059", "label": "film production company"},
		{"qid": "Q4220917", "label": "film award"},
		{"qid": "Q618779", "label": "award"},
		{"qid": "Q47461344", "label": "written work"},
		{"qid": "Q15773347", "label": "film character"},
		{"qid": "Q104649845", "label": "film organization"}
	],
	"relations": [
		{"pid": "P57", "label": "director", "domain": "Q11424", "range": "Q5"},
		{"pid": "P58", "label": "screenwriter", "domain": "Q11424", "range": "Q5"},
		{"pid": "P136", "label": "genre", "domain": "Q11424", "range": "Q483394"},
		{"pid": "P144", "label": "based on", "domain": "Q11424", "range": "Q47461344"},
		{"pid": "P161", "label": "cast member", "domain": "Q11424", "range": "Q5"},
		{"pid": "P166", "label": "award received", "domain": "Q11424", "range": "Q618779"},
		{"pid": "P272", "label": "production company", "domain": "Q11424", "range": "Q1762059"},
		{"pid": "P495", "label": "country of origin", "domain": "Q11424", "range": "Q6256"},
		{"pid": "P577", "label": "publication date", "domain": "Q11424", "range": ""},
		{"pid": "P674", "label": "characters", "domain": "Q11424", "range": "Q15773347"},
		{"pid": "P840", "label": "narrative location", "domain": "Q11424", "range": "Q515"},
		{"pid": "P915", "label": "filming location", "domain": "Q11424", "range": "Q515"},
		{"pid": "P921", "label": "main subject", "domain": "Q11424", "range": ""},
		{"pid": "P1411", "label": "nominated for", "domain": "Q11424", "range": "Q618779"},
		{"pid": "P2130", "label": "cost", "domain": "Q11424", "range": ""}
	]
}

Возьмем онтологию из репозитория по кинофильмам. Если мы посмотрим на структуру, то заметим пустые значения в ключе relations. Что подразумевает некую вариантивность доступных типов значений для категории объекта в триплетах. Более того, если углубиться в содержание триплетов из указанного репозитория, то мы можем обнаружить некоторые несоответствия в эталонных данных, которые ставят под сомнение истинность в разметке. Например, для типа связи "publication date" эталонных триплеты содержат информацию, которая не была отражена в "сыром" тексте. Ниже несколько примеров таких коллизий:

1. {"id": "ont_1_movie_test_1", "sent": "Bleach: Hell Verse (Japanese: BLEACH , Hepburn: Bur\u00c4\u00abchi Jigoku-Hen) is a 2010 Japanese animated film directed by Noriyuki Abe.", "triples": [{"sub": "Bleach : Hell Verse", "rel": "director", "obj": "Noriyuki Abe"}, {"sub": "Bleach : Hell Verse", "rel": "publication date", "obj": "01 January 2010"}]}

В этом примере в изначальном тексте указан год, при этом в эталонном триплете дата расширена для формата дд-мм-гггг, и может не отражать фактическую дату выхода фильма.

2. {"id": "ont_1_movie_test_6", "sent": "The Life on the Earth - The Summer of Dioxin ( Inochi No Chikyuu - Dioxin No Natsu), also known as Tracing the Gray Summer, is a 2001 Japanese anime drama film by Satoshi Dezaki, a joint production between the Magic Bus and the GoGo Visual Planning which recounts the true story of the Seveso disaster, a chemical incident occurred over the Italian town of Seveso in 1976, which is still considered one of the worst ecological disasters in history.", "triples": [{"sub": "Inochi No Chikyuu : Dioxin No Natsu", "rel": "director", "obj": "Satoshi Dezaki"}, {"sub": "Inochi No Chikyuu : Dioxin No Natsu", "rel": "publication date", "obj": "00  2001"}, {"sub": "Inochi No Chikyuu : Dioxin No Natsu", "rel": "based on", "obj": "Seveso disaster"}]}

В этом примере дата в эталонном триплете сложно интерпретируема.

3. {"id": "ont_1_movie_test_17", "sent": "Who Framed Roger Rabbit is a 1988 American live-action/animated mystery comedy film directed by Robert Zemeckis, produced by Frank Marshall and Robert Watts, and written by Jeffrey Price and Peter S. Seaman.", "triples": [{"sub": "Who Framed Roger Rabbit", "rel": "director", "obj": "Robert Zemeckis"}, {"sub": "Who Framed Roger Rabbit", "rel": "publication date", "obj": "22 June 1988"}, {"sub": "Who Framed Roger Rabbit", "rel": "publication date", "obj": "02 December 1988"}, {"sub": "Who Framed Roger Rabbit", "rel": "publication date", "obj": "27 October 1988"}, {"sub": "Who Framed Roger Rabbit", "rel": "screenwriter", "obj": "Jeffrey Price"}, {"sub": "Who Framed Roger Rabbit", "rel": "screenwriter", "obj": "Peter S. Seaman"}]}


В этом примере для одной и той же сущности в эталонных триплетах указаны несколько связей с разными датами. При этом в изначальном тексте дата публикации не конкретизирована до дня и месяца.

С учетом вышесказанного в рамках настоящего эксперимента мы исключим из онтологии и разменных данных те типы связей, которые подразумевают неоднозначность интерпретации для целей более качественной оценки экстракции информации при генерировании триплетов различными моделями.

In [3]:
ont = {
	"title": "Movie Ontology",
	"id": "ont_1_movie",
	"concepts": [
		{"qid": "Q5", "label": "human"},
		{"qid": "Q515", "label": "city"},
		{"qid": "Q6256", "label": "country"},
		{"qid": "Q11424", "label": "film"},
		# {"qid": "Q201658", "label": "film genre"},  Неиспользуемая сущность в онтологии
		{"qid": "Q483394", "label": "genre"},
		{"qid": "Q1762059", "label": "film production company"},
		# {"qid": "Q4220917", "label": "film award"}, Неиспользуемая сущность в онтологии
		{"qid": "Q618779", "label": "award"},
		{"qid": "Q47461344", "label": "written work"},
		{"qid": "Q15773347", "label": "film character"},
		# {"qid": "Q104649845", "label": "film organization"} Неиспользуемая сущность в онтологии
	],
	"relations": [
		{"pid": "P57", "label": "director", "domain": "Q11424", "range": "Q5"},
		{"pid": "P58", "label": "screenwriter", "domain": "Q11424", "range": "Q5"},
		{"pid": "P136", "label": "genre", "domain": "Q11424", "range": "Q483394"},
		{"pid": "P144", "label": "based on", "domain": "Q11424", "range": "Q47461344"},
		{"pid": "P161", "label": "cast member", "domain": "Q11424", "range": "Q5"},
		{"pid": "P166", "label": "award received", "domain": "Q11424", "range": "Q618779"},
		{"pid": "P272", "label": "production company", "domain": "Q11424", "range": "Q1762059"},
		{"pid": "P495", "label": "country of origin", "domain": "Q11424", "range": "Q6256"},
		{"pid": "P674", "label": "characters", "domain": "Q11424", "range": "Q15773347"},
		{"pid": "P840", "label": "narrative location", "domain": "Q11424", "range": "Q515"},
		{"pid": "P915", "label": "filming location", "domain": "Q11424", "range": "Q515"},
		{"pid": "P1411", "label": "nominated for", "domain": "Q11424", "range": "Q618779"},
	]
}

Объединяем типы данных в relations по id.

In [4]:
concepts_dict = {x['qid']: x['label'] for x in ont['concepts']}
relations_with_text = [{**relation, 'domain_label': concepts_dict.get(relation['domain']), 'range_label': concepts_dict.get(relation['range'])} for relation in ont['relations']]
print(relations_with_text)

[{'pid': 'P57', 'label': 'director', 'domain': 'Q11424', 'range': 'Q5', 'domain_label': 'film', 'range_label': 'human'}, {'pid': 'P58', 'label': 'screenwriter', 'domain': 'Q11424', 'range': 'Q5', 'domain_label': 'film', 'range_label': 'human'}, {'pid': 'P136', 'label': 'genre', 'domain': 'Q11424', 'range': 'Q483394', 'domain_label': 'film', 'range_label': 'genre'}, {'pid': 'P144', 'label': 'based on', 'domain': 'Q11424', 'range': 'Q47461344', 'domain_label': 'film', 'range_label': 'written work'}, {'pid': 'P161', 'label': 'cast member', 'domain': 'Q11424', 'range': 'Q5', 'domain_label': 'film', 'range_label': 'human'}, {'pid': 'P166', 'label': 'award received', 'domain': 'Q11424', 'range': 'Q618779', 'domain_label': 'film', 'range_label': 'award'}, {'pid': 'P272', 'label': 'production company', 'domain': 'Q11424', 'range': 'Q1762059', 'domain_label': 'film', 'range_label': 'film production company'}, {'pid': 'P495', 'label': 'country of origin', 'domain': 'Q11424', 'range': 'Q6256', 'd

Список именованных сущностей из онтологии

In [5]:
concepts_list = [x['label'] for x in ont['concepts']]
concepts = ', '.join(concepts_list)
concepts

'human, city, country, film, genre, film production company, award, written work, film character'

Список комбинаций связей в именованных сущностях

In [6]:
relations_list = []
for i in relations_with_text:
    rel = f"sub: {i['domain_label']}, rel: {i['label']}, obj: {i['range_label']}\n"
    relations_list.append(rel)
relations_schema = "".join(relations_list)
print(relations_schema)

sub: film, rel: director, obj: human
sub: film, rel: screenwriter, obj: human
sub: film, rel: genre, obj: genre
sub: film, rel: based on, obj: written work
sub: film, rel: cast member, obj: human
sub: film, rel: award received, obj: award
sub: film, rel: production company, obj: film production company
sub: film, rel: country of origin, obj: country
sub: film, rel: characters, obj: film character
sub: film, rel: narrative location, obj: city
sub: film, rel: filming location, obj: city
sub: film, rel: nominated for, obj: award



Автор репозитория стремится получить ответ в формате relation(subject, object), что вынуждает добавить дополнительную предобработку выводов модели для дальнейшего использовании их при построении графа знаний. Модифицируем подход, чтобы модель возвращала триплеты в формале [subject, relation, object]. Более того, автор репозитория пытается за один проход решить комплексную задачу: определить сущности и определить тип связи. При решении комплексных задач слабые модели зачасую склонны к галлюцинациям и демонстрируют низкое качество генерации. В настоящем исследовании мы пойдем по пути декомпозиции задачи, предполагая, что слабые модели смогут демонстрировать более лучшее качество при решении отдельных небольших задач.

При этом автор репозитория использует динамическое промптирование, подставляя в промпт примеры из наиболее похожих примеров в обучающей выборке, что делает сам подход довольно негибким. В настоящем исследовании предлагаю использовать более универсальный подход. Он будет состоять из двух этапов:

1. На первом этапе мы получаем из необработанного текста все именованные сущности из онтологии (схемы).
2. Получаем через модель наиболее подходящие связи из найденных моделью именованных сущностей, формируя таким образом триплеты в соответствии с обозначенной онтологией.

Референс: https://bratanic-tomaz.medium.com/constructing-knowledge-graphs-from-text-using-openai-functions-096a6d010c17

В данной статье добавлен дополнительных шаг для поиска скрытых дубликатов в именованных сущностях. Для данных из репозитория это не особо релевантно, поскольку каждый текст обособлен и предположительно тексты не пересекаются по содержанию

Создаем объект схемы для вычленения именованных сущностей из списка concepts. Для каждой сущности мы прописываем краткое описание и предоставляем пример подстроки, в которой содержится эта сущность. Более того, под капотом такого метода происходит парсинг вывода модели, что позволяет работать сразу с выводом модели, как со словарем. Такой подход предположительно должен хорошо работать на моделях OpenAI, которые обученые на большых корпусах данных и хорошо понимают промпты, предполагающие получение вывода в формате Json. Локальные модели могут демонстрировать посредственное качество особенно при определении связей между сущностями, но при проведении файнтьюнинга можно добавить более высокого качества генерации.

Референс: https://medium.com/@Gouse_MG/streamline-your-data-extraction-and-analysis-with-langchain-kor-gpt-a962b4be05ca 

In [7]:
schema = Object(
    id="movie",
    description="information about a movie",
    attributes=[
        Text(
            id="human",
            description="The name of the human: director, screenwriter, cast member, etc.",
            examples=[("film directed by Noriyuki Abe", "Noriyuki Abe")],
        ),
        Text(
            id="city",
            description="The city where the movie was filmed or the city in which the film takes place",
            examples=[("a Vienna-based company", "Vienna")],
        ),
        Text(
            id="country",
            description="The country where the movie was filmed or the country in which the film takes place",
            examples=[("The story takes place in Clairton, Pennsylvania,", "Pennsylvania")],
        ),
        Text(
            id="film",
            description="The name of the film",
            examples=[("Bleach: Hell Verse (Japanese: BLEACH", "Bleach: Hell Verse")],
        ),
        Text(
            id="genre",
            description="The genre of the film",
            examples=[("Japanese animated fantasy film", "fantasy film")]
        ),
        Text(
            id="film_production_company",
            description="The name of the film production company",
            examples=[("short film produced by Pixar", "Pixar")]
        ),
        Text(
            id="award",
            description="The name of the award",
            examples=[("The film won the National Film Award for Best Feature Film", "National Film Award for Best Feature Film")]
        ),
        Text(
            id="written_work",
            description="The written work on which the film is based",
            examples=[("Adventures of Nils Holgersson is based on the novel The Wonderful Adventures of Nils", "The Wonderful Adventures of Nils")]
        ),
        Text(
            id="film_character",
            description="The name of the film character",
            examples=[("the first creation of the character Mickey Mouse", "Mickey Mouse")]
        ),
    ],
    examples=[
            (
                "Welcome to Eltingville was written by Dorkin and Chuck Sheetz and received a positive reception.",
                [
                    {"human": "Dorkin", "human": "Chuck Sheetz", "written work": "Welcome to Eltingville"},
                ],
            ),
            (
                "Mobile Suit Gundam F91 was Gundam creator Yoshiyuki Tomino's attempt to launch a new Gundam saga, set 30 years after Char's Counterattack and 27 years after the later published Gundam Unicorn.",
                [
                    {"human": "Yoshiyuki Tomino", "film": "Mobile Suit Gundam F91"},
                ],
            )
    ],
    many=True,
)

## Инициализируем модель 

Первоначальная идея потестить разные модели разбилась о суровую реальность возможностей и времени

Поэтому ниже просто изначальный пример - сама работа дальше по ноутбуками по номерам

In [1]:
model_path = "../models/zephyr-7b-beta.Q6_K.gguf"
model_path = "../models/mistral-7b-v0.1.Q5_K_M.gguf"
model_path = "../models/llama-2-13b-chat.Q5_K_M.gguf"
model_path = "../models/openchat-3.5-0106.Q8_0.gguf"
# model_path = "../models/mixtral-8x7b-v0.1.Q6_K.gguf"

In [9]:
llm = LlamaCpp(
    model_path=model_path,
    temperature=0,
    max_tokens=256,
    n_gpu_layers= 40,
    n_batch = 16,
    n_ctx=2048,
)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/openchat-3.5-0106.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-0106
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.att

#### Создает цепочку для экстракции именованных сущностей

In [10]:
ner_chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

In [11]:
res = ner_chain.invoke("Mobile Suit Gundam F91 was Gundam creator Yoshiyuki Tomino's attempt to launch a new Gundam saga, set 30 years after Char's Counterattack and 27 years after the later published Gundam Unicorn.")


llama_print_timings:        load time =    7967.55 ms
llama_print_timings:      sample time =      20.34 ms /   256 runs   (    0.08 ms per token, 12586.04 tokens per second)
llama_print_timings: prompt eval time =   12615.57 ms /   885 tokens (   14.25 ms per token,    70.15 tokens per second)
llama_print_timings:        eval time =    6743.44 ms /   255 runs   (   26.44 ms per token,    37.81 tokens per second)
llama_print_timings:       total time =   19731.07 ms /  1140 tokens


In [12]:
entities = res['text']['data']['movie']
entities

[{'human': 'Yoshiyuki Tomino', 'film': 'Mobile Suit Gundam F91'}]

#### Создает собственный промпт для определения типа связи между сущностями

In [13]:
GET_RELATIONS_TEMPLATE = """You are a networked intelligence helping a human track knowledge triples.
You are given a list of NAMED ENTITIES and a LIST OF POSIBLE RELATIONS. You are to identify the relations between the named entities. 
Do not use any other named entities or relations. You MUST use only provided NAMED ENTITIES and LIST OF POSIBLE RELATIONS. 

LIST OF POSIBLE RELATIONS: {relations}

NAMED ENTITIES: {entities}

To indetidy the relations between named entities, you can use the following context: {context}

Format your response as a JSON. Do not add any attributes that do not appear in the schema shown below:

    {{
    "triplets": [
        {{
        "sub": "string",
        "rel": "string",
        "obj": "string"
        }}
    ]
    }}

Do NOT add any clarifying information. Output MUST follow the schema above. Do NOT add any additional columns that do not appear in the schema.
Answer:"""

In [14]:
relation_prompt = PromptTemplate.from_template(GET_RELATIONS_TEMPLATE)

In [15]:
rel_chain = relation_prompt | llm | JsonOutputParser()

In [16]:
context = "Mobile Suit Gundam F91 was Gundam creator Yoshiyuki Tomino's attempt to launch a new Gundam saga, set 30 years after Char's Counterattack and 27 years after the later published Gundam Unicorn."

In [17]:
res = rel_chain.invoke({"relations": relations_schema, "entities": entities, "context": context})

Llama.generate: prefix-match hit

llama_print_timings:        load time =    7967.55 ms
llama_print_timings:      sample time =      21.73 ms /   256 runs   (    0.08 ms per token, 11780.41 tokens per second)
llama_print_timings: prompt eval time =    2684.05 ms /   489 tokens (    5.49 ms per token,   182.19 tokens per second)
llama_print_timings:        eval time =    6520.89 ms /   255 runs   (   25.57 ms per token,    39.11 tokens per second)
llama_print_timings:       total time =    9569.18 ms /   744 tokens


In [18]:
print(res)

{'triplets': [{'sub': 'Mobile Suit Gundam F91', 'rel': 'director', 'obj': 'Yoshiyuki Tomino'}, {'sub': 'Mobile Suit Gundam F91', 'rel': 'screenwriter', 'obj': 'Yoshiyuki Tomino'}, {'sub': 'Mobile Suit Gundam F91', 'rel': 'genre', 'obj': 'Gundam'}, {'sub': 'Mobile Suit Gundam F91', 'rel': 'based on', 'obj': "Char's Counterattack"}, {'sub': 'Mobile Suit Gundam F91', 'rel': 'cast member', 'obj': 'Yoshiyuki Tomino'}]}


Оформим обозначенные выше шаги в класс

In [36]:
class TripletBuilder:
    models_dict = {
        1: "../models/zephyr-7b-beta.Q6_K.gguf",
        2: "../models/mistral-7b-v0.1.Q5_K_M.gguf",
        3: "../models/llama-2-13b-chat.Q5_K_M.gguf",
        4: "../models/openchat-3.5-0106.Q8_0.gguf",
        5: "../models/mixtral-8x7b-v0.1.Q6_K.gguf"
    }    

    def __init__(self, relations: str, models: int=5, n_gpu_layers: int = 50, model_path: str = None, schema=schema, relation_prompt=relation_prompt, n_runs: int =50):
        """
        Initialize the TripletBuilder class.

        Args:
            relations (str): The relations to extract triplets for.
            models (int, optional): The number of models to choose from. Defaults to 5.
            n_gpu_layers (int, optional): The number of GPU layers. Defaults to 50.
            model_path (str, optional): The path to the model. Defaults to None.
            schema (str, optional): The schema for named entity recognition. Defaults to schema.
            relation_prompt (str, optional): The prompt for relation extraction. Defaults to relation_prompt.
            n_runs (int, optional): The number of runs to perform. Defaults to 50.
        """
        if model_path:
            self.model_path = model_path
        else:
            self.model_path = self.models_dict[models]
        self.schema = schema
        self.relations = relations
        self.relation_prompt = relation_prompt
        self.n_runs = n_runs

        self.llm = LlamaCpp(
            model_path=self.model_path,
            temperature=0,
            max_tokens=256,
            n_gpu_layers= n_gpu_layers,
            n_batch = 16,
            n_ctx=2048,
        )

        self.ner_chain = create_extraction_chain(self.llm, self.schema, encoder_or_encoder_class='json')

        self.relations_chain = self.relation_prompt | self.llm | JsonOutputParser()

    def load_data(self, file_path: str):
        """
        Load data from a file.

        Args:
            file_path (str): The path to the file.
        """
        with open(file_path, 'r') as f:
            self.data = f.readlines()

    def get_triplets(self):
        """
        Extract triplets from the data and return a list of dictionaries containing the model name, sentence, and triplets.

        Returns:
            list: A list of dictionaries, each containing the model name, sentence, and triplets.
        """
        result = []

        for i_text in tqdm(self.data[:self.n_runs]):
            i_text = json.loads(i_text)
            temp_dict = dict()

            context = i_text['sent']
            try:
                entities = self.ner_chain.invoke(context)['text']['data']['movie']
            except:
                entities = []

            print(context)
            print("++++++++++++++++++++++++++++++++++++")
            print(entities)
            print("++++++++++++++++++++++++++++++++++++")

            try:
                if len(entities[0]) < 2:
                    triplets= { "triplets": []}
                else:
                    try:
                        triplets = self.relations_chain.invoke({"relations": self.relations, "entities": entities, "context": context})
                    except:
                        triplets= { "triplets": []}
            except:
                triplets= { "triplets": []}

            print(triplets)

            temp_dict['model'] = os.path.basename(self.model_path)
            temp_dict['sent'] = context
            temp_dict['triples'] = triplets['triplets']

            print("====================================")
            print(temp_dict)
            print("====================================")

            result.append(temp_dict)

        return result


In [62]:
builder = TripletBuilder(relations_schema, models=4, n_runs=100)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from ../models/openchat-3.5-0106.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = openchat_openchat-3.5-0106
llama_model_loader: - kv   2:                       llama.context_length u32              = 8192
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.att

Указывам путь к jsonl файлу из репозитория для получения из него текстов для экстракции данных

In [63]:
builder.load_data('./movie_ground_truth.jsonl')

In [64]:
test = builder.get_triplets()

  0%|          | 0/100 [00:00<?, ?it/s]
llama_print_timings:        load time =     194.16 ms
llama_print_timings:      sample time =      25.72 ms /   256 runs   (    0.10 ms per token,  9953.34 tokens per second)
llama_print_timings: prompt eval time =    4723.60 ms /   871 tokens (    5.42 ms per token,   184.39 tokens per second)
llama_print_timings:        eval time =    6791.33 ms /   255 runs   (   26.63 ms per token,    37.55 tokens per second)
llama_print_timings:       total time =   12055.91 ms /  1126 tokens
Llama.generate: prefix-match hit


Bleach: Hell Verse (Japanese: BLEACH , Hepburn: BurÄ«chi Jigoku-Hen) is a 2010 Japanese animated film directed by Noriyuki Abe.
++++++++++++++++++++++++++++++++++++
[{'film': 'Bleach: Hell Verse', 'human': 'Noriyuki Abe'}]
++++++++++++++++++++++++++++++++++++



llama_print_timings:        load time =     194.16 ms
llama_print_timings:      sample time =      21.27 ms /   256 runs   (    0.08 ms per token, 12034.60 tokens per second)
llama_print_timings: prompt eval time =    2491.22 ms /   473 tokens (    5.27 ms per token,   189.87 tokens per second)
llama_print_timings:        eval time =    6513.35 ms /   255 runs   (   25.54 ms per token,    39.15 tokens per second)
llama_print_timings:       total time =    9372.03 ms /   728 tokens
  1%|          | 1/100 [00:21<35:22, 21.44s/it]Llama.generate: prefix-match hit


{'triplets': [{'sub': 'Bleach: Hell Verse', 'rel': 'director', 'obj': 'Noriyuki Abe'}]}
{'model': 'openchat-3.5-0106.Q8_0.gguf', 'sent': 'Bleach: Hell Verse (Japanese: BLEACH , Hepburn: BurÄ«chi Jigoku-Hen) is a 2010 Japanese animated film directed by Noriyuki Abe.', 'triples': [{'sub': 'Bleach: Hell Verse', 'rel': 'director', 'obj': 'Noriyuki Abe'}]}



llama_print_timings:        load time =     194.16 ms
llama_print_timings:      sample time =      21.89 ms /   256 runs   (    0.09 ms per token, 11695.37 tokens per second)
llama_print_timings: prompt eval time =    4603.79 ms /   870 tokens (    5.29 ms per token,   188.97 tokens per second)
llama_print_timings:        eval time =    6745.21 ms /   255 runs   (   26.45 ms per token,    37.80 tokens per second)
llama_print_timings:       total time =   11730.66 ms /  1125 tokens
Llama.generate: prefix-match hit


...

{'triplets': [{'sub': 'Jerry, Jerry, Quite Contrary', 'rel': 'director', 'obj': 'Chuck Jones'}, {'sub': 'Jerry, Jerry, Quite Contrary', 'rel': 'screenwriter', 'obj': 'Chuck Jones'}, {'sub': 'Jerry, Jerry, Quite Contrary', 'rel': 'production company', 'obj': 'Chuck Jones'}]}
{'model': 'openchat-3.5-0106.Q8_0.gguf', 'sent': 'Jerry, Jerry, Quite Contrary is a 1966 Tom and Jerry short written, produced and directed by Chuck Jones.', 'triples': [{'sub': 'Jerry, Jerry, Quite Contrary', 'rel': 'director', 'obj': 'Chuck Jones'}, {'sub': 'Jerry, Jerry, Quite Contrary', 'rel': 'screenwriter', 'obj': 'Chuck Jones'}, {'sub': 'Jerry, Jerry, Quite Contrary', 'rel': 'production company', 'obj': 'Chuck Jones'}]}



llama_print_timings:        load time =     194.16 ms
llama_print_timings:      sample time =      22.34 ms /   256 runs   (    0.09 ms per token, 11459.78 tokens per second)
llama_print_timings: prompt eval time =    4534.82 ms /   864 tokens (    5.25 ms per token,   190.53 tokens per second)
llama_print_timings:        eval time =    6786.96 ms /   255 runs   (   26.62 ms per token,    37.57 tokens per second)
llama_print_timings:       total time =   11702.70 ms /  1119 tokens
Llama.generate: prefix-match hit


The short is directed by William Hanna and Joseph Barbera, composed by Scott Bradley, and animated by Ray Patterson, Irven Spence, Pete Burness, and Kenneth Muse.
++++++++++++++++++++++++++++++++++++
[{'human': 'Joseph Barbera', 'film_production_company': 'MGM'}]
++++++++++++++++++++++++++++++++++++



llama_print_timings:        load time =     194.16 ms
llama_print_timings:      sample time =      22.86 ms /   256 runs   (    0.09 ms per token, 11199.58 tokens per second)
llama_print_timings: prompt eval time =    2496.14 ms /   466 tokens (    5.36 ms per token,   186.69 tokens per second)
llama_print_timings:        eval time =    6545.91 ms /   255 runs   (   25.67 ms per token,    38.96 tokens per second)
llama_print_timings:       total time =    9397.36 ms /   721 tokens
100%|██████████| 100/100 [30:03<00:00, 18.03s/it]

{'triplets': [{'sub': 'Joseph Barbera', 'rel': 'director', 'obj': 'film'}, {'sub': 'Joseph Barbera', 'rel': 'screenwriter', 'obj': 'film'}, {'sub': 'Joseph Barbera', 'rel': 'production company', 'obj': 'MGM'}]}
{'model': 'openchat-3.5-0106.Q8_0.gguf', 'sent': 'The short is directed by William Hanna and Joseph Barbera, composed by Scott Bradley, and animated by Ray Patterson, Irven Spence, Pete Burness, and Kenneth Muse.', 'triples': [{'sub': 'Joseph Barbera', 'rel': 'director', 'obj': 'film'}, {'sub': 'Joseph Barbera', 'rel': 'screenwriter', 'obj': 'film'}, {'sub': 'Joseph Barbera', 'rel': 'production company', 'obj': 'MGM'}]}


In [65]:
test

[{'model': 'openchat-3.5-0106.Q8_0.gguf',
  'sent': 'Bleach: Hell Verse (Japanese: BLEACH , Hepburn: BurÄ«chi Jigoku-Hen) is a 2010 Japanese animated film directed by Noriyuki Abe.',
  'triples': [{'sub': 'Bleach: Hell Verse',
    'rel': 'director',
    'obj': 'Noriyuki Abe'}]},
 {'model': 'openchat-3.5-0106.Q8_0.gguf',
  'sent': "Keyboard Cat's original form was a video originally made in 1984 by Charlie Schmidt of his cat Fatso seemingly playing a piano (though manipulated by Schmidt off-camera) to a cheery tune.",
  'triples': [{'sub': 'Keyboard Cat',
    'rel': 'director',
    'obj': 'Charlie Schmidt'},
   {'sub': 'Keyboard Cat', 'rel': 'screenwriter', 'obj': 'Charlie Schmidt'},
   {'sub': 'Keyboard Cat', 'rel': 'genre', 'obj': 'comedy'},
   {'sub': 'Keyboard Cat', 'rel': 'based on', 'obj': 'real life'},
   {'sub': 'Keyboard Cat', 'rel': 'cast member', 'obj': 'Fatso'},
   {'sub': 'Keyboard Cat',
    'rel': 'production company',
    'obj': 'Charlie Schmidt'},
   {'sub': 'Keyboard Ca

Сохраняем полученный файл

In [66]:
with open("./mixtral_movie_full.jsonl", 'w', encoding='utf-8') as f:
    for entry in test:
        json_record = json.dumps(entry, ensure_ascii=False)
        f.write(json_record + '\n')

### Промежуточные выводы по генерации триплетов

1. Глобально при генерации триплетов можно пойти двумя путями:
    * Экстрации всех возможных триплетов без онтологии. Такой подход реализован в библиотеках langchain и llamaindex. Это самый простой подход, достаточно только указать разделитель триплетов для их последующего прасинга. При этом наиблучшее качество экстракции ожидаемо показывают модели OpenAI. При использовании каких-либо локальных моделей качество экстракции ниже и, как правило, зависит от размера модели.
    * Экстрация триплетов с использованием онтологии (схемы). В этом подходе можно попробовать через промтп добиваться экстрации сущностей и связей в один проход обращения к LLM. В ходе экспериментов с этим хорошо могут справиться модели OpenAI, одна локальные опенсорс модели демонстрируют значительно более низкое качество. Чтобы добиться более выского качества генерации триплетов, мы идем по пути декомпозции задач: идентификация сущностей и последующий поиск подходящих связей между ними изходя из схемы онтологии.

2. Основные проблемы:
    * Одна из главных проблем при работе с LLM, это обработка выводом модели. То есть нам важно одновременно добиться стабильного ожидаемого формата вывода модели и продумать алгоритм постобработки этого вывода. Одним из наиболее подходящих способов - это добиться от модели стрктурирвоанного ответа в формате JSON. В некоторых библиотеках это уже реализовано путем промпт-инжиниринга при решении типовых задач, как, например, задача NER. При решении более нетривиальных задач приходится добавиться структурированного выводв путем написания собственных промтов. 

    В рамках данного проекта для решения задачи NER использую библиотеку KOR https://eyurtsev.github.io/kor/tutorial.html
    В отличие от коробочного решения из langchain, KOR предоставляет более гибкие инструменты для описания сущностей и few-shot промтинга. Одновременно с этим мы получаем от модели сразу десериализованный вывод, с которым нам на много проще потом работать на следующих этапах пайплайна. 

    Для определения связи между сущностями используется уже кастомный промпт, который позволяет добавить сразу необходимого нам формата [sub, rel, obj], который мы может без дополнительных постобработок добавить для построениея QA-чейна с использованием графа знаний в RAG.

    * Здесь сразу стоит отметить, что при решении задач на всех моделях установлено значение temperature 0. Это необходимо для достижения максимального детерминированного поведения модели, и является общей рекомендацией при решении задач экстракции и создании QA-систем с RAG. 


## примеры промптов

In [2]:
example = """<|user|>
Context: Visions of Light film genre is Science and Documentary
</s>
<|assistant|>
(Visions of Light, genre, Science)<|>(Visions of Light, genre, Documentary)
</s>
<|user|>
Context: The Visions of Light film screenplay was done by Quentin Tarantino and John Travolta
</s>
<|assistant|>
(Visions of Light, screenplay, Quentin Tarantino)<|>(Visions of Light, screenplay, John Travolta)
</s>"""

In [22]:
TRIPLE_EXTRACTION_TEMPLATE = """<|system|>
You are a networked intelligence helping a human track knowledge triples about ONTOLOGY RELATIONS between ONTOLOGY CONCEPTS in provided context.
A knowledge triple must contain only three elements. 
A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.
Note that the context may contain only some of the Ontology relations between Ontology Concepts.
Use only information provided in context. Do not use own knowledge.
Ontology Concepts: {concepts}
ONTOLOGY RELATIONS: {relations}
</s>
{example}
<|user|>
Context: {text}
</s>
<|assistant|>"""

In [47]:
example = """Context: Visions of Light film genre is Science and Documentary
Answer:(Visions of Light, genre, Science)<|>(Visions of Light, genre, Documentary)

Context: The Visions of Light film screenplay was done by Quentin Tarantino and John Travolta
Answer:(Visions of Light, screenplay, Quentin Tarantino)<|>(Visions of Light, screenplay, John Travolta)"""

In [66]:
TRIPLE_EXTRACTION_TEMPLATE_M = """You are a networked intelligence helping a human track knowledge triples about ONTOLOGY RELATIONS between ONTOLOGY CONCEPTS in provided context.
A knowledge triple must contain only three elements. 
A knowledge triple is a clause that contains a subject, a predicate, and an object. The subject is the entity being described, the predicate is the property of the subject that is being described, and the object is the value of the property.
Note that the context may contain only some of the Ontology relations between Ontology Concepts.
Use only information provided in context. Do not use own knowledge.
Ontology Concepts: {concepts}
ONTOLOGY RELATIONS: {relations}


Context: {text}
Answer:"""

In [ ]:
prompt = """
You are a language model trained to perform Named Entity Recognition (NER) on text data.
Given a sentence or a paragraph, your task is to identify and classify named entities into predefined categories such as person names, organizations, locations, dates, etc.

Example:
Input: "Apple Inc. is planning to open a new store in New York City next month."
Output: {"entities": [{"text": "Apple Inc.", "label": "ORG"}, {"text": "New York City", "label": "LOC"}]}

Please provide the text you want to perform NER on:
"""
